<a href="https://colab.research.google.com/github/thomasamged/brain-tumor-class/blob/main/brain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
navoneel_brain_mri_images_for_brain_tumor_detection_path = kagglehub.dataset_download('navoneel/brain-mri-images-for-brain-tumor-detection')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob as gb
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
import matplotlib.pyplot as  plt
from tensorflow.keras.layers import Conv2D,Flatten,Dense,Dropout,MaxPool2D
from tensorflow.keras.models import Sequential
from imblearn.over_sampling import RandomOverSampler
import warnings

In [ ]:
su=['jpg','JPG','jpeg']
size_no=[]
for s in su:
    for i in gb.glob("/kaggle/input/brain-mri-images-for-brain-tumor-detection/no"+f"/*.{s}"):
        image=cv2.imread(i)
        size_no.append(image.shape)
print(pd.Series(size_no).value_counts(),"\n",pd.Series(size_no).shape)

In [ ]:
su=['jpg','JPG','jpeg']
size_yes=[]
for s in su:
    for i in gb.glob("/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes"+f"/*.{s}"):
        image=cv2.imread(i)
        size_yes.append(image.shape)
print(pd.Series(size_yes).value_counts(),"\n",pd.Series(size_yes).shape)

In [ ]:
size=128

In [ ]:
su=['jpg','JPG','jpeg']
X_yes=[]
for s in su:
    for i in gb.glob("/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes"+f"/*.{s}"):
        image=cv2.imread(i)
        image=cv2.resize(image,(size,size))
        X_yes.append(image)

In [ ]:
sizesu=['jpg','JPG','jpeg']
X_no=[]
for s in su:
    for i in gb.glob("/kaggle/input/brain-mri-images-for-brain-tumor-detection/no"+f"/*.{s}"):
        image=cv2.imread(i)
        image=cv2.resize(image,(size,size))
        X_no.append(image)


In [ ]:
def get_name(code):
    if code==0:
        return "no"
    return "yes"

In [ ]:
X=[]
y=[]
for i in X_yes:
    X.append(i)
    y.append(1)
for i in X_no:
    X.append(i)
    y.append(0)
print(len(X))


In [ ]:
X=np.array(X)
y=np.array(y)


In [ ]:
plt.figure(figsize=(20,20))
for n,i in enumerate(list(np.random.randint(0,len(X),36))):
    plt.subplot(6,6,n+1)
    plt.imshow(X[i])
    plt.axis("off")
    plt.title(get_name(y[i]))

In [ ]:
X=X.astype("float32")
X/=255

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.3,shuffle=True,stratify=y)

In [ ]:
brain=Sequential([
    Conv2D(32,kernel_size=(3,3),activation="relu",input_shape=(size,size,3)),
    MaxPool2D(2,2),
    Conv2D(64,kernel_size=(3,3),activation="relu"),
    MaxPool2D(2,2),
    Flatten(),
    Dense(100,activation="relu"),
    Dropout(rate=.5),
    Dense(1,activation="sigmoid")])
brain.compile(optimizer="adam",loss='binary_crossentropy',metrics=["accuracy"])

In [ ]:

brain.summary()


In [ ]:
warnings.filterwarnings("ignore")
history=brain.fit(X_train,y_train,epochs=30
                  ,verbose=1,batch_size=10)

In [ ]:
pre=brain.predict(X_test)
pr=[]
for i in pre:
    if i<.5:
        pr.append(0)
    else:
        pr.append(1)
print(accuracy_score(y_test,pr))

In [ ]:
loss,acc=brain.evaluate(X_test,y_test)
print(loss,acc)

In [ ]:
print(classification_report(y_test,pr))

In [ ]:
plt.figure(figsize=(20,20))
for n,i in enumerate(list(np.random.randint(0,len(X_test),36))):
    plt.subplot(6,6,n+1)
    plt.imshow(X_test[i])
    plt.axis("off")
    plt.title(f"actual {get_name(y_test[i])} | predicted {get_name(pr[i])}")